In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from colormath.color_objects import sRGBColor, XYZColor
from colormath.color_conversions import convert_color
from scipy.optimize import minimize
import scipy.interpolate as intp
import scipy.integrate as integrate
from scipy.optimize import newton
matplotlib.use('Qt5Agg')
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
MF = np.genfromtxt("CMF_5nm.csv", delimiter=',')
CMF = MF[:, 1:4]

In [3]:
def clip2rgb(xyz):
    if type(xyz) is not XYZColor:
        xyz = XYZColor(xyz[0], xyz[1], xyz[2])
    crgb = convert_color(xyz, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    return [r, g, b]

In [4]:
def g_euc(r1, r2):
    return np.sqrt((r1[0]-r2[0])**2 + (r1[1]-r2[1])**2 + (r1[2]-r2[2])**2)

In [5]:
def g_line2point(ln, pt):
    # distance between point and line
    # line ~ (a,b,c) coefficients of ax + by + c = 0
    # point ~ (x0, y0)
    return abs(ln[0]*pt[0] + ln[1]*pt[1] + ln[2])/np.sqrt(ln[0]**2 + ln[1]**2)

In [6]:
def character(cmf_vector, wtpt):
    """Characteristic Color of pseudo-wavelength-index i
        observe that the characteristic color is the closest point to the white point
    """
    O = cmf_vector
    # define line in 3d space with xvec, yvec, zvec
    xvec = np.linspace(0, 60, 500)
    yvec = (O[1]/O[0])*(xvec-O[0]) + O[1]
    zvec = (O[2]/O[0])*(xvec-O[0]) + O[2]
    rvec = [np.array([xvec[i], yvec[i], zvec[i]]) for i, _ in enumerate(xvec)]
    distset = [g_euc(r, wtpt) for r in rvec]
    idx = np.argmin(distset)
    return rvec[idx]

In [7]:
def volume(a,b,c):
    # https://en.wikipedia.org/wiki/Tetrahedron#Volume
    # black point: (0,0,0)
    # volume: |a * (b x c) | / 6
    return abs(a.dot(np.cross(b, c))) / 6

In [8]:
def get_partition(xspline, yspline, zspline, wtpt, trials, N):
    # 1. split spline into a bunch of tiny points
    vec = lambda s: np.array([xspline(s), yspline(s), zspline(s)])
    input_pts = np.linspace(0, 2*np.pi, trials + 1)
    spline_pts = vec(input_pts)
    # 2. calculate the volume for each wedge
    volumes = [volume(wtpt, spline_pts[:, i], spline_pts[:, i+1]) for i in range(trials)]
    # 3. fitting routine to get the equal partitions
    target = sum(volumes)/N
    new_pts = [0]
    new_volumes = []
    i = 0
    while i < len(volumes):
        v = 0
        while v < target and i < len(volumes):
            v += volumes[i]
            i += 1
        new_pts.append(i)
        new_volumes.append(v)
    return new_pts, new_volumes, spline_pts[:, new_pts]
    

In [9]:
def mensurate(cmf, abeam, N): 
    # make appended unit vector array from CMF
    wtpt = cmf.T@abeam
    ucmf = [c/np.linalg.norm(c) for c in cmf]
    coeffs = np.linspace(0, 1, 30)
    c1 = cmf[-1]/np.linalg.norm(cmf[-1])
    c0 = cmf[0]/np.linalg.norm(cmf[0])
    mixture = []
    for c in coeffs:
        mix = c*c0 + (1-c)*c1
        mixture.append(mix/np.linalg.norm(mix))
    units = np.concatenate((ucmf, mixture))
    
    # find characteristic color of each unit
    xvec = np.zeros(len(units)+1)
    yvec = np.zeros(len(units)+1)
    zvec = np.zeros(len(units)+1)
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    for idx, u in enumerate(units):
        c = character(u, wtpt) 
        xvec[idx] = c[0]
        yvec[idx] = c[1]
        zvec[idx] = c[2]
        ax.scatter(u[0], u[1], u[2], color=clip2rgb(u))
        ax.scatter(c[0], c[1], c[2], color=clip2rgb(c))
       
        
    # make the spline periodic 
    xvec[-1] = xvec[0]
    yvec[-1] = yvec[0]
    zvec[-1] = zvec[0]
        
    
    # fit 3 pre-mensuration splines
    I = np.linspace(0, 2*np.pi, len(units)+1)
    x_spline = intp.make_interp_spline(I, xvec)
    y_spline = intp.make_interp_spline(I, yvec)
    z_spline = intp.make_interp_spline(I, zvec)
   

    # get equal spline pts
    _, _, equal_spline_pts = get_partition(x_spline, y_spline, z_spline, wtpt, trials=int(1e6), N=100)
    for s in equal_spline_pts.T:
        ax.scatter(s[0], s[1], s[2], color='black', s=5)
    
    # find total volume of the double cone 
    vec = lambda s: np.array([x_spline(s), y_spline(s), z_spline(s)])
    area = lambda s: (np.linalg.norm(np.cross(vec(s), wtpt)))**2
    V = integrate.quad(area, 0, 2*np.pi, limi=1000)[0]
    
    interv = np.linspace(0, 1, 100)
    for idx, i in enumerate(interv):
        ax.scatter(x_spline(i), y_spline(i), z_spline(i))
        ax.text(x_spline(i), y_spline(i), z_spline(i), str(idx))
    
    
    # find intervales that divide V into equal parts
    dV = V/N
    pts = [0]
    deltaV = lambda s0, s1: area(s0)*g_euc(vec(s0), vec(s1))/2
    root = lambda s0, s1, dV: deltaV(s0, s1) - dV

    for n in range(N):
        s0 = newton(root, x0 = pts[n], args=[pts[n], dV], maxiter=1000)
        pts.append(s0)
        
    print(pts)
        
    

In [10]:
abeam = np.ones(CMF.shape[0])
M = mensurate(CMF, abeam, 100)

TypeError: quad() got an unexpected keyword argument 'limi'